## Course Work


In [3]:
!pip install -q transformers[torch]
!pip install -q datasets
!pip install -q accelerate -U
!pip install -q py7zr
!pip install -q evaluate nltk rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import evaluate
from torch import nn
import math

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Load Datasets
from google.colab import files
uploaded = files.upload()

# Create datasets
train_dataset = pd.read_csv('MTS-Dialog-TrainingSet.csv')
test_dataset = pd.read_csv('MTS-Dialog-ValidationSet.csv')

## Prepare dataset
#train_data = train_dataset.rename(columns={'context': 'input_text', 'target': 'target_text'})
train_data = train_dataset.rename(columns={
    'dialogue': 'input_text',      # Doctor-patient conversation
    'section_text': 'target_text'  # Structured clinical notes
})

# Convert to Hugging Face Dataset
#dataset = Dataset.from_pandas(train_data)
dataset = Dataset.from_pandas(train_data[['input_text', 'target_text']])  # Only keep needed columns

In [11]:
#model_name = 'facebook/opt-2.7b'
#model = OPTForCausalLM.from_pretrained(model_name)
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def preprocess_function(examples):
    # Prefix the input to guide the model
    inputs = [f"Summarize medical dialogue: {text}" for text in examples['input_text']]
    targets = examples['target_text']

    # Tokenize inputs
    model_inputs = tokenizer(
        inputs,
        max_length=256,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=256,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



In [12]:
# Tokenize dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)

Map:   0%|          | 0/1201 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [13]:
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)

# Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Train
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bixjnn (bixjnn-n-a) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,16.227200
20,16.071400
30,14.881500
40,15.218900
50,14.682900
60,13.186100
70,12.460900
80,11.073100
90,10.757800
100,9.309900


TrainOutput(global_step=301, training_loss=5.912362241665786, metrics={'train_runtime': 476.5565, 'train_samples_per_second': 2.52, 'train_steps_per_second': 0.632, 'total_flos': 81272751783936.0, 'train_loss': 5.912362241665786, 'epoch': 1.0})

In [14]:
# Function to generate text
def generate_text(model, tokenizer, input_text):
    # Generation prompt
    full_input = f"Generate a restaurant description: {input_text}"

    # Tokenize the input
    inputs = tokenizer(
        full_input,
        return_tensors="pt",
        max_length=256,
        truncation=True,
        padding="max_length"
    )

    # Move inputs to model device
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Generate text
    outputs = model.generate(
        **inputs,
        max_length=256,
        num_beams=5,
        early_stopping=True
    )

    # Decode the generated text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [15]:
def generate_medical_summary(model, tokenizer, dialogue_text):
    """
    Generates a structured medical summary from doctor-patient dialogue.

    Args:
        model: Fine-tuned T5 model
        tokenizer: T5 tokenizer
        dialogue_text: Raw doctor-patient conversation text

    Returns:
        Structured clinical note with headers (cc, hpi, etc.)
    """
    # Medical summarization prompt
    prompt = f"Summarize this medical dialogue into structured clinical notes: {dialogue_text}"

    # Tokenize input
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        max_length=512,  # Increased for medical dialogues
        truncation=True,
        padding="max_length"
    ).to(model.device)

    # Generate summary
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=512,  # Match input length
        num_beams=4,     # Balance between quality and speed
        early_stopping=True,
        repetition_penalty=2.5  # Reduce repetition in clinical notes
    )

    # Decode and clean output
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Post-processing for clinical format
    return format_clinical_notes(summary)

def format_clinical_notes(raw_summary):
    """
    Ensures consistent formatting of clinical note sections.
    Example: Ensures each header starts on a new line.
    """
    # Add newlines before headers if missing
    headers = ["cc:", "hpi:", "pmh:", "assessment:", "plan:"]
    for header in headers:
        raw_summary = raw_summary.replace(header, "\n" + header)

    return raw_summary.strip()

In [17]:
# Load test dataset
test_dataset = pd.read_csv('MTS-Dialog-ValidationSet.csv')

# Prepare for text generation
import torch
model.eval()
torch.cuda.empty_cache()

In [21]:
# Generate predictions
predictions = []
for dialogue in test_dataset['dialogue']:  # Changed from 'context' to 'dialogue'
    with torch.no_grad():
        # Updated prompt to use the correct column name
        input_text = f"Summarize this medical dialogue: {dialogue}"
        prediction = generate_text(model, tokenizer, input_text)
        predictions.append(prediction)

# Add predictions to the test dataset
test_dataset = test_dataset.add_column('prediction', predictions)

AttributeError: 'DataFrame' object has no attribute 'add_column'

In [18]:
# Generate predictions
predictions = []
for context in test_dataset['context']:
    with torch.no_grad():
        prediction = generate_medical_summary(model, tokenizer, context)
        predictions.append(prediction)

# Add predictions to the test dataset
test_dataset['prediction'] = predictions

KeyError: 'context'

In [ ]:
# Prepare final CSV
submission_df = test_dataset[['context', 'prediction']]

# Save final CSV
submission_df.to_csv('final.csv', index=False)

print("Submission CSV generated successfully!")

# Test
print("\nSample Predictions:")
print(submission_df.head())

Submission CSV generated successfully!

Sample Predictions:
                                             context  \
0  name[The Wrestlers], eatType[pub], food[Japane...   
1  name[The Vaults], eatType[pub], food[Japanese]...   
2  name[Giraffe], eatType[restaurant], food[Fast ...   
3      name[Cocum], eatType[pub], near[The Sorrento]   
4  name[The Phoenix], eatType[restaurant], food[F...   

                                          prediction  
0  The Wrestlers is a family friendly pub located...  
1  The Vaults is a Japanese pub located in the ci...  
2  Giraffe is a family friendly restaurant locate...  
3                  Cocum is a pub near The Sorrento.  
4  The Phoenix is a family friendly restaurant lo...  
